In [4]:
import pandas as pd
import jieba
import jieba.analyse
import jieba.posseg as pseg
from snownlp import SnowNLP
import operator

In [5]:
import orangecontrib.associate.fpgrowth as oaf

In [10]:
import synonyms

ImportError: DLL load failed: 找不到指定的模块。

In [12]:
#'读入数据'
df = pd.read_csv('C:/Users/GWT/Desktop/adas+comments.csv',encoding="gbk")

UnicodeDecodeError: 'gbk' codec can't decode byte 0x80 in position 5: incomplete multibyte sequence

In [ ]:
#'筛选，只要淘宝的，不要天猫的'
df_taobao = df[df['source'] == 'taobao']

'去掉默认评价'
df_taobao = df_taobao[(df_taobao['comments'] != '评价方未及时做出评价,系统默认好评!') & 
                      (df_taobao['comments'] != '15天内买家未作出评价') & 
                      (df_taobao['comments'] != '此用户没有填写评价。')]

In [ ]:
'分类，好评、差评'
df_good = df_taobao[df_taobao['grade'] == 1]
df_bad = df_taobao[df_taobao['grade'] == -1]

In [ ]:
'提取评论'
comments_good = df_good['comments']
comments_bad = df_bad['comments']

In [ ]:
'保存成txt格式'
comments_good.to_csv('C:/Users/yangxing/Desktop/taobao/comments_good.txt', index=False)
comments_bad.to_csv('C:/Users/yangxing/Desktop/taobao/comments_bad.txt', index=False)

In [ ]:
'读取txt文件'
file_name1 = "C:/Users/yangxing/Desktop/taobao/comments_good.txt"
file_name2 = "C:/Users/yangxing/Desktop/taobao/comments_bad.txt"
content = open(file_name1, "r").read()
s = SnowNLP(content).sentences

In [ ]:
'结巴分词并提取形容词和名词'
fh = open('C:/Users/yangxing/Desktop/taobao/noun.txt', 'w')
for i in range(len(s)):
    words = pseg.cut(s[i])
    for word, flag in words:
        if flag=='a' or flag=='n':
            fh.write(word)
            fh.write(',')
    fh.write('\n')
fh.close()

In [ ]:
'去除多余的逗号和换行符'
file_name3 = "C:/Users/yangxing/Desktop/taobao/noun.txt"
noun = open(file_name3, "r").read()
noun1=noun.replace(',\n','\n').replace('\n\n','\n').replace('\n\n','\n').replace('\n\n','\n').replace('\n\n','\n').replace('\n\n','\n')
fh = open('C:/Users/yangxing/Desktop/taobao/noun1.txt', 'w')
fh.write(noun1)
fh.close()

In [ ]:
'以列表形式读入'
noun1 = open('C:/Users/yangxing/Desktop/taobao/noun1.txt', "r").read()
arr = noun1.split('\n')

'去掉最后一个空行'
arr = arr[:-1]

'将列表内部由字符串形式转为列表形式'
for i in range(len(arr)):
    arr[i]=arr[i].split(',')

'关联分析找出频繁项集'
supportRate = 0.003  #支持度
itemsets = dict(oaf.frequent_itemsets(arr, supportRate))  #频繁项集
itemsets2={k: v for k, v in itemsets.items() if len(k)>=2}   #元素项大于等于2

'筛选出既包含名词、又包含形容词的频繁项集'
dict_an={}
for i in range(len(list(itemsets2.keys()))):
    m=0
    n=0
    fro = list(itemsets2.keys())[i]
    sup = list(itemsets2.values())[i]
    for j in fro:
        k=pseg.cut(j)
        for word,flag in k:
           if flag == 'a':m=1
           if flag == 'n':n=1
    if m+n==2:
        #print(fro,':',sup)
        dict_an[fro]=sup

'列举频繁项集的单个元素项'
array=[]
for i in range(len(list(dict_an.keys()))):
    fro = list(dict_an.keys())[i]
    for j in fro:
        array.append(j)
        
array = list(set(array))  #去重

'对单个元素项进行同义词合并'
synos=['0']
for i in range(len(array)):
    for j in range(len(synos)):
        if synonyms.compare(array[i], synos[j], seg=True)>0.8:
            break
        else:
            if j == (len(synos)-1):
                synos.append(array[i])
synos.remove('0')

'同义词替代+去重'
sy_dict={'0':0}
for i in range(len(list(dict_an.keys()))):
    fro = list(dict_an.keys())[i]
    sup = list(dict_an.values())[i]
    fro_an = list(fro)[0:len(list(fro))]
    for j in range(len(fro_an)):
        for k in range(len(synos)):
            if synonyms.compare(fro_an[j], synos[k], seg=True)>0.8:
                fro_an[j]=synos[k]
                break
    for m in range(len(sy_dict)):
        if frozenset(fro_an) == list(sy_dict.keys())[m]:
            sy_dict[list(sy_dict.keys())[m]] = list(sy_dict.values())[m] + sup
            break
        else:
            if m==(len(sy_dict)-1):
                sy_dict[frozenset(fro_an)]=sup
del sy_dict['0']